# Import Modules

In [2]:
# iPython Settings
%load_ext autoreload
%autoreload 2

# Python Modules
import sys
sys.path.insert(0, "..")

import copy

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from ase.visualize import view

import plotly.plotly as py
import plotly.graph_objs as go

# My Modules ******************************************************************
from dft_job_automat.job_analysis import DFT_Jobs_Analysis
from dft_job_automat.job_types_classes.dft_methods import DFT_Methods
from orr_reaction.orr_methods import ORR_Free_E_Plot, calc_ads_e
from orr_reaction.orr_methods import lowest_e_path, df_calc_adsorption_e, plot_all_states
from energetics.dft_energy import Element_Refs

from colors.colors import color_list

# Local Imports
from an_data_processing import load_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Divisor is Energy class instance!!!


# Script Methods

In [3]:
h2o_fe_corr = 0.013
h2_fe_corr = -0.035

def calculate_gibbs_correction(row, h2o_corr, h2_corr):
    adsorbate = row["adsorbate"]
    ads_corr = row["gibbs_correction"]

    corr_tot = "NaN"
    if adsorbate == "ooh":
        corr_tot = ads_corr - 2 * h2o_corr + (3. / 2.) * h2_corr 
    elif adsorbate == "o":
        corr_tot = ads_corr - 1 * h2o_corr + (2. / 2.) * h2_corr
    elif adsorbate == "oh":
        corr_tot = ads_corr - 1 * h2o_corr + (1. / 2.) * h2_corr
    elif adsorbate == "h2o":
        corr_tot = ads_corr - 1 * h2o_corr + (0. / 2.) * h2_corr

    return(corr_tot)

# Dataframe Processing

In [4]:
df = load_df(from_file=True)
df["new__"] = df.apply(calculate_gibbs_correction, args=(h2o_fe_corr, h2_fe_corr), axis=1)

col_list = [
    'new__',
    'ads_e',
    'adsorbate',

#     'atom_type_num_dict',
#     'atoms_object',
#     'charge_history',

#     'elec_energy',  # <-------------------
    
    'final_site',

#     'full_path',

    'gibbs_correction',

#     'graph_site',  # <-------------------

#     'init_atoms',
#     'magmom_history',
#     'max_revision',
    
    'notes',  # <-------------------

#     'path',
#     'pdos_data',
#     'revision_number',
    'site',
    'spinpol',
    'system',
#     'tree_level_labels',
#     'variable_list',
    ]

df = df[col_list]

df["spinpol"] = df.apply(
    lambda row: False if row["system"] == "N_graph_Fe" and np.isnan(row["spinpol"]) else row["spinpol"],
    axis=1
    )

df_master = df.copy()

df_2 = df_master.copy()
df_2 = df_2[np.isfinite(df_2['new__'])]
df_2 = df_2[df_2["gibbs_correction"] != 0.]

Attempting to load df from pickle


# Free Energy Correction Bar Chart

In [5]:
master_data_list = []

grouped = df_2.groupby(["system", "site", "spinpol"])
for group_i in grouped:
    group_name = group_i[0]
    df_i = group_i[1]
    
    # Setting Variable Color List
    color_list = 5 * ["#006600"]
    notes_i = df_i["notes"].tolist()
    
   
    for i_ind, note_j in enumerate(notes_i):
        if note_j == "Desorbed":
#             color_list[i_ind] = "rgb(204, 0, 102, 0.3)"
            color_list[i_ind] = "rgb(255,204,204)"
        elif note_j == "Dissociated":
            color_list[i_ind] = "rgb(255,255,204)"

    # Setting Variable Border Color
    b_color_list = 5 * ["black"]
    notes_i = df_i["notes"].tolist()

    sys_i = df_i["system"].iloc[0]
    if sys_i == "Fe_slab":
#         b_color = "rgb(0,255,0)"
        b_color = "#6600CC"
    elif sys_i == "graphene":
        b_color =  "black"
    elif sys_i == "graph_Fe":
        b_color =  "brown"
    elif sys_i == "N_graph_Fe":
        b_color =  "red"
            
    data = go.Bar(
        x=df_i["adsorbate"],
        y=df_i["new__"],
        name=str(group_name),
#         opacity=[0.3, 0.2, 0.1, 0.6, 1.],
        marker=dict(
            color=color_list,
            line=dict(
                color=b_color,
                width=3,
                )
            ),
        )

    master_data_list = master_data_list + [data]

In [6]:
# Plotting
layout = dict(
    title = 'Free-energy Corrections',
#     xaxis=dict(
#         title="ORR Species",
#         ),
    yaxis=dict(
        title="Free-energy Correction [eV]",
        ),

    barmode="group",
#     bargap=0.25,
    bargap=0.2,
    bargroupgap=0.4,

    legend=dict(
#         x=0.3,
#         y=0.,
        traceorder='normal',
        orientation="h",
        font=dict(
#             family='sans-serif',
            size=8,
#             color='#000'
        ),
#         bgcolor='#E2E2E2',
#         bordercolor='#FFFFFF',
        borderwidth=1
        ),
    )   

fig = dict(data=master_data_list, layout=layout)
py.iplot(fig, filename='01_fe_graph/pl_fe_corr_orr')

Aw, snap! We didn't get a username with your request.

Don't have an account? https://plot.ly/api_signup

Questions? accounts@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.

In [ ]:
grouped = df_master.groupby(["adsorbate"])
for group_i in grouped:
    group_name = group_i[0]
    df_i = group_i[1]
    
    df_i = df_i[df_i["notes"] != "Dissociated"]
    df_i = df_i[df_i["notes"] != "Desorbed"]
    df_i = df_i[np.isfinite(df_i['new__'])]
    df_i = df_i[df_i["gibbs_correction"] != 0.]

    mean_fe_corr_i = df_i["new__"].mean()
    
    print(group_name, ":", str(mean_fe_corr_i))

The equation used depends on whether the the value of  
$V_{max}$ is R, G, or B.  

$$H←  ​​​​​0 ​+​ \frac{​​30(G−B)​​}{Vmax−Vmin}  ​​, if V​max​​ = R$$

$$c = \sqrt{a^2 + b^2}$$

In [ ]:
%%latex

\section{First section}
\textbf{Hello}


\begin{equation} 
c = \sqrt{a^2 + b^2}
\end{equation}


\begin{itemize}
\item First item
\item Second item
\end{itemize}
\textbf{World}